# Experiment 2 run files

Create counterbalanced run files for each of 20 participants in fMRI experiment 2 and save to "exp_2/runs" directory. Run this in choose-stimuli conda environment.

In [11]:
import sys
import os
import numpy as np
import pandas as pd

from itertools import permutations

sketch_morph_path = os.getcwd()

function for making balanced latin squares. Adapted from [Paul Grau's demo](https://medium.com/@graycoding/balanced-latin-squares-in-python-2c3aa6ec95b9) on Medium

In [12]:
def balanced_latin_squares(n):
    l = [[((j//2+1 if j%2 else n-j//2) + i) % n + 1 for j in range(n)] for i in range(n)]
    if n % 2:  # Repeat reversed for odd n
        l += [seq[::-1] for seq in l]
    return l

Create 4 runs of part 1 (sketch_ID) for each of 20 subjects

In [13]:
objects = ['fixation',
           'pig_alarm-clock','hedgehog_bush','hand_cactus','face_radio',
           'face_strawberry','foot_hockey-stick','rabbit_scissors','lion_sun']

trial_order = balanced_latin_squares(9)
StimType_base = ['sketch','question',
                 'sketch','question','sketch','question','sketch','question','sketch','question',
                 'sketch','question','sketch','question','sketch','question','sketch','question']
block_durations = [8,2,8,2,8,2,8,2,8,2,8,2,8,2,8,2,8,2]

for subject in range(1,21):
    sub_objects = np.random.permutation(objects)
    order_count = 0

    runs = range(1,5)
    for run in runs:
        subDF = pd.DataFrame()
        block_start = 0

        StimNo = []
        Onsets = []
        where_animate = []
        Jitter = 0.5 + np.random.uniform(low=-.2,high=.2, size=72)
        trial_objects_run = []
        for block_no in range(1,5):
            block_onsets = [block_start+4, block_start+14,
                            block_start+16,block_start+26,
                            block_start+28,block_start+38,
                            block_start+40,block_start+50,
                            block_start+52,block_start+62,
                            block_start+64,block_start+74,
                            block_start+76,block_start+86,
                            block_start+88,block_start+98,
                            block_start+100,block_start+110]

            block_start +=108
            Onsets.append(block_onsets)

            trial_objects_block = []
            trial_right_left = np.random.permutation([1,1,1,1,0,0,0,0])
            right_left_count = 0
            for trial in range(9):
                morph = sub_objects[trial_order[order_count%18][trial]-1]        
                trial_objects_block.append(morph)

                if morph == 'fixation':
                    StimNo.append(-1)
                    where_animate.append('None')
                    trial_objects_block.append('fixation')
                    where_animate.append('None')
                    StimNo.append(-1)
                else:
                    StimNo.append(block_no+3)
                    where_animate.append('None')
                    trial_objects_block.append(morph)
                    StimNo.append(block_no+3)

                    if trial_right_left[right_left_count] == 1:
                        where_animate.append('left')
                    elif trial_right_left[right_left_count] == 0: 
                        where_animate.append('right')
                    else:
                         print('where_animate must be either 0 (right) or 1 (left)')       

                    right_left_count+=1
            order_count+=1
            trial_objects_run.append(trial_objects_block)

        StimType = np.tile(StimType_base,4)
        Onsets_flat = [onset for block_onset_list in Onsets for onset in block_onset_list];
        trial_objects_run_flat = [stim for blocklist in trial_objects_run for stim in blocklist];

        subDF['ObjectID'] = trial_objects_run_flat
        subDF['Onset'] = Onsets_flat
        subDF['Jitter'] = Jitter
        subDF['StimNo'] = StimNo
        subDF['WhereAnimate'] = where_animate
        subDF['Duration'] = np.tile(block_durations,4)
        subDF['StimType'] = StimType
        subDF['StimType'][subDF['ObjectID']=='fixation'] = 'fixation'

        subDF.to_csv(os.path.join(sketch_morph_path,'runs','Sub{:02d}_Run{:02d}'.format(subject,run)+'.csv')) 


/anaconda3/envs/choose-stim/lib/python3.5/site-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
# check out subject 20, run 4
subDF

,ObjectID,Onset,Jitter,StimNo,WhereAnimate,Duration,StimType
0,face_radio,4,0.509165,4,None,8,sketch
1,face_radio,14,0.417093,4,left,2,question
2,foot_hockey-stick,16,0.429050,4,None,8,sketch
3,foot_hockey-stick,26,0.310829,4,left,2,question
4,hedgehog_bush,28,0.458743,4,None,8,sketch
5,hedgehog_bush,38,0.378493,4,left,2,question
6,face_strawberry,40,0.654410,4,None,8,sketch
7,face_strawberry,50,0.633585,4,right,2,question
8,fixation,52,0.478215,-1,None,8,fixation
9,fixation,62,0.575749,-1,None,2,fixation


Create 12 runs of part 2 (ambisketch) for each of the 20 subjects. Order of block instructions (animate/inanimate/neutral) counterbalanced across runs. 

In [16]:
objects = ['pig_alarm-clock','hedgehog_bush','hand_cactus','face_radio',
           'face_strawberry','foot_hockey-stick','rabbit_scissors','lion_sun']

trial_order = balanced_latin_squares(8)

StimType_base = ['fixation','INSTRUCTIONS_REPLACE','PREPARE_REPLACE',
                 'sketch','question','sketch','question','sketch','question','sketch','question',
                 'sketch','question','sketch','question','sketch','question','sketch','question']
block_durations = [8,2,8,8,2,8,2,8,2,8,2,8,2,8,2,8,2,8,2]

instruct_order = permutations(['Animate','Inanimate','Neutral'])

for subject in range(1,21):
    sub_objects = np.random.permutation(objects)
    sub_stim_blocktype_idx = np.random.permutation([0,1,2])
    order_count = 0

    # figure out order for stimulus exemplar presentation
    sub_block_stim_order = []
    block_stim_order = []
    [block_stim_order.append(perm) for perm in permutations([1,2,3])];

    sub_block_stim_order.extend(block_stim_order)
    sub_block_stim_order.extend(block_stim_order)

    # figure out order for animate, inanimate, neutral blocks in each run
    sub_instruct_order = []
    instruct_order = []
    [instruct_order.append(perm) for perm in permutations(['Animate','Inanimate','Neutral'])];
    sub_instruct_order.extend(instruct_order)
    sub_instruct_order.extend(instruct_order)

    runs = range(5,17)
    for run in runs:
        subDF = pd.DataFrame()
        block_start = 0
        StimNo = []
        Onsets = []
        StimType_run = []
        where_correct = []
        Jitter = 0.5 + np.random.uniform(low=-.2,high=.2, size=57)
        trial_objects_run = []
        for block_no in range(3):
            StimType_block = []
            block_onsets = [block_start, block_start+8, block_start+10,
                            block_start+18,block_start+26,
                            block_start+28,block_start+36,
                            block_start+38,block_start+46,
                            block_start+48,block_start+56,
                            block_start+58,block_start+66,
                            block_start+68,block_start+76,
                            block_start+78,block_start+86,
                            block_start+88,block_start+96]

            Onsets.append(block_onsets)
            block_start +=98

            StimType_block.extend(StimType_base)

            block_type = sub_instruct_order[run-5][block_no]
            StimType_block[1] = 'Instruct_{}'.format(block_type)
            StimType_block[2] = 'Prepare_{}'.format(block_type)
            StimType_run.append(StimType_block) 

            trial_objects_block = ['fixation','instructions','prepare']
            where_correct.append('None')
            where_correct.append('None')
            where_correct.append('None')
            StimNo.append(-1)
            StimNo.append(-1)
            StimNo.append(-1)

            if block_type == 'Animate':
                stim_number = sub_block_stim_order[run-5][sub_stim_blocktype_idx[0]]
            elif block_type == 'Inanimate':
                stim_number = sub_block_stim_order[run-5][sub_stim_blocktype_idx[1]]
            elif block_type == 'Neutral':
                stim_number = sub_block_stim_order[run-5][sub_stim_blocktype_idx[2]]
            else:
                print('block type must be Animate, Inanimate, or Neutral...')

            trial_right_left = np.random.permutation([1,1,1,1,0,0,0,0])
            right_left_count = 0

            for trial in range(8):
                morph = sub_objects[trial_order[order_count%8][trial]-1]
                trial_objects_block.append(morph)
                StimNo.append(stim_number)

                if block_type == 'Inanimate':
                    object_1 = morph.split('_')[0]
                    object_2 = morph.split('_')[1]
                    trial_objects_block.append(object_2+'_'+object_1)
                else:
                    trial_objects_block.append(morph)

                StimNo.append(-1)
                where_correct.append('None')

                if trial_right_left[right_left_count] == 1:
                    where_correct.append('left')
                elif trial_right_left[right_left_count] == 0: 
                    where_correct.append('right')
                else:
                     print('where_correct must be either 0 (right) or 1 (left)')       

                right_left_count+=1

            order_count+=1
            trial_objects_run.append(trial_objects_block)

        Onsets_flat = [onset for block_onset_list in Onsets for onset in block_onset_list];
        trial_objects_run_flat = [stim for blocklist in trial_objects_run for stim in blocklist];
        StimType_run_flat = [item for type_list in StimType_run for item in type_list];

        subDF['ObjectID'] = trial_objects_run_flat
        subDF['Onset'] = Onsets_flat
        subDF['Jitter'] = Jitter
        subDF['StimNo'] = StimNo
        subDF['WhereCorrect'] = where_correct
        subDF['Duration'] = np.tile(block_durations,3)
        subDF['StimType'] = StimType_run_flat
        subDF['StimType'][subDF['ObjectID']=='fixation'] = 'fixation'

        subDF.to_csv(os.path.join(sketch_morph_path,'runs','Sub{:02d}_Run{:02d}'.format(subject,run)+'.csv')) 



/anaconda3/envs/choose-stim/lib/python3.5/site-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
# check out subject 20, run 16
subDF

,ObjectID,Onset,Jitter,StimNo,WhereCorrect,Duration,StimType
0,fixation,0,0.475270,-1,None,8,fixation
1,instructions,8,0.462798,-1,None,2,Instruct_Neutral
2,prepare,10,0.323947,-1,None,8,Prepare_Neutral
3,foot_hockey-stick,18,0.570112,1,None,8,sketch
4,foot_hockey-stick,26,0.495456,-1,right,2,question
5,face_radio,28,0.539667,1,None,8,sketch
6,face_radio,36,0.303137,-1,left,2,question
7,lion_sun,38,0.469135,1,None,8,sketch
8,lion_sun,46,0.587901,-1,right,2,question
9,rabbit_scissors,48,0.507325,1,None,8,sketch
